In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = " ", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.12.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import kagglehub
path = kagglehub.dataset_download("thedevastator/mental-health-chatbot-pairs")

Using Colab cache for faster access to the 'mental-health-chatbot-pairs' dataset.


In [ ]:
print(path)

/kaggle/input/mental-health-chatbot-pairs


In [ ]:
import pandas as pd
import os
os.listdir(path)
# print(df.head())
df = pd.read_csv(os.path.join(path, "train.csv"))
print(df.head())

df.to_csv("train.csv", index= False)   #to load the csv

                                                text
0  <HUMAN>: What is a panic attack?\n<ASSISTANT>:...
1  <HUMAN>: What are symptoms of panic attack vs....
2  <HUMAN>: What are the types of Mental Illness?...
3  <HUMAN>: What does mental-illness mean?\n<ASSI...
4  <HUMAN>: How can you treat mental illness?\n<A...


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files="train.csv",
    split="train"
)


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.column_names

['text']

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    texts= []
    for conv in examples["text"]:
      if "<HUMAN>:" in conv and "<ASSISTANT>:" in conv:
        human = conv.split("<HUMAN>:")[1].split("<ASSISTANT>:")[0].strip()
        assistant = conv.split("<ASSISTANT>:")[1].strip()


        instruction = "You are a supportive and empathetic mental health assistant.Accordingly answer the questions asked"
        input       = human
        output      = assistant

          # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
        return { "text" : texts, }
pass


dataset = dataset.map(formatting_prompts_func, remove_columns=dataset.column_names)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/172 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 172 | Num Epochs = 3 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.677100
2,1.366600
3,1.632400
4,1.487900
5,1.683700
6,1.199400
7,1.435000
8,1.489700
9,1.633800
10,1.017800


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a supportive and empathetic mental health assistant.Accordingly answer the questions asked ", # instruction
        "How to manage stress?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a supportive and empathetic mental health assistant.Accordingly answer the questions asked \n\n### Input:\nHow to manage stress?\n\n### Response:\n1. Identify the stressors in your life and try to eliminate or reduce them whenever possible. This might involve making changes to your lifestyle, such as getting more exercise, eating a healthy diet, or reducing your workload.\n\n2. Practice relaxation techniques, such as meditation, deep breathing exercises, or yoga. These can help']

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
model.push_to_hub("Full-Finetuned-Llama3.1-8B")
tokenizer.push_to_hub("Full-Finetuned-Llama3.1-8B")

README.md:   0%|          | 0.00/605 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          |  557kB /  168MB            

Saved model to https://huggingface.co/Full-Finetuned-Llama3.1-8B


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpv5olvy4_/tokenizer.json:   2%|1         |  295kB / 17.2MB            

In [ ]:
model.push_to_hub_merged(
    "prranjal/16-bit-Llama3.1-8B",
    tokenizer,
    save_method="merged_16bit"
)

config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...lama3.1-8B/tokenizer.json:  97%|#########7| 16.7MB / 17.2MB            

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0004-of-00004.safetensors:   3%|2         | 33.5MB / 1.17GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [14:25<00:00, 216.43s/it]


Unsloth: Merge process complete. Saved to `/content/prranjal/16-bit-Llama3.1-8B`
